In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from Model import *

In [2]:
tf.random.set_seed(7)

In [ ]:
#model_list = [cnn1,cnn2, cnn3,cnn4,cnn5,cnn6, cnn8, cnn9, cnn10]
model_list = [lstm6]
for model_obj in model_list:
    model = model_obj.Model(vocabulary2)
    print(model.name)
    model.model.summary()
    if not os.path.exists(f"logs/{model.name}"):
        os.mkdir(f"logs/{model.name}")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"logs/{model.name}")
    model.run_experiment([train['review_text'], train['n_comments'], train['n_votes']], train['rating'], epochs=30, tensorboard_callback=tensorboard_callback, batch_size=23)

lstm6
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 1400)        0           ['input_1[0][0]']                
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 1400, 32)     1600000     ['text_vectorization[0][0]']     
                                                                                                  
 bidirectional (Bidirectional)  (None, 1400, 64)     12672       ['embedding[0][0]']    

In [3]:
train = pd.read_csv("../../dataset/goodreads_train.csv")
vocabulary2 = np.load('../../vocabulaires/voc_without_std_word_count_5.npy', allow_pickle=True)[:50000]

In [5]:
model.run_experiment([train['review_text'], train['n_comments'], train['n_votes']], train['rating'], epochs=5, batch_size=100)

Epoch 1/5
7200/7200 [==============================] - 2432s 338ms/step - loss: 1.1402 - categorical_accuracy: 0.5574 - val_loss: 1.1219 - val_categorical_accuracy: 0.5644
Epoch 2/5
7200/7200 [==============================] - 2405s 334ms/step - loss: 1.1352 - categorical_accuracy: 0.5589 - val_loss: 1.0965 - val_categorical_accuracy: 0.5703
Epoch 3/5
7200/7200 [==============================] - 2388s 332ms/step - loss: 1.1286 - categorical_accuracy: 0.5616 - val_loss: 1.1076 - val_categorical_accuracy: 0.5680
Epoch 4/5
7200/7200 [==============================] - 2374s 330ms/step - loss: 1.1238 - categorical_accuracy: 0.5639 - val_loss: 1.1135 - val_categorical_accuracy: 0.5698
Epoch 5/5
7200/7200 [==============================] - 2378s 330ms/step - loss: 1.1177 - categorical_accuracy: 0.5672 - val_loss: 1.1016 - val_categorical_accuracy: 0.5698


In [6]:
model.run_experiment([train['review_text'], train['n_comments'], train['n_votes']], train['rating'], epochs=5, batch_size=100)

Epoch 1/5
7200/7200 [==============================] - 2393s 332ms/step - loss: 1.1138 - categorical_accuracy: 0.5688 - val_loss: 1.1079 - val_categorical_accuracy: 0.5692
Epoch 2/5
7200/7200 [==============================] - 2378s 330ms/step - loss: 1.1094 - categorical_accuracy: 0.5710 - val_loss: 1.0996 - val_categorical_accuracy: 0.5728
Epoch 3/5
7200/7200 [==============================] - 2423s 337ms/step - loss: 1.1033 - categorical_accuracy: 0.5732 - val_loss: 1.1029 - val_categorical_accuracy: 0.5725
Epoch 4/5
7200/7200 [==============================] - 2415s 335ms/step - loss: 1.0995 - categorical_accuracy: 0.5744 - val_loss: 1.0974 - val_categorical_accuracy: 0.5769
Epoch 5/5
7200/7200 [==============================] - 2407s 334ms/step - loss: 1.0954 - categorical_accuracy: 0.5753 - val_loss: 1.0859 - val_categorical_accuracy: 0.5777


In [7]:
model.run_experiment([train['review_text'], train['n_comments'], train['n_votes']], train['rating'], epochs=15, batch_size=100)

Epoch 1/15
7200/7200 [==============================] - 2401s 333ms/step - loss: 1.0907 - categorical_accuracy: 0.5773 - val_loss: 1.0850 - val_categorical_accuracy: 0.5822
Epoch 2/15
7200/7200 [==============================] - 2372s 329ms/step - loss: 1.0857 - categorical_accuracy: 0.5792 - val_loss: 1.1049 - val_categorical_accuracy: 0.5740
Epoch 3/15
7200/7200 [==============================] - 2378s 330ms/step - loss: 1.0810 - categorical_accuracy: 0.5809 - val_loss: 1.1267 - val_categorical_accuracy: 0.5646
Epoch 4/15
7200/7200 [==============================] - 2374s 330ms/step - loss: 1.0763 - categorical_accuracy: 0.5827 - val_loss: 1.0881 - val_categorical_accuracy: 0.5822
Epoch 5/15
7200/7200 [==============================] - 2372s 329ms/step - loss: 1.0719 - categorical_accuracy: 0.5842 - val_loss: 1.0753 - val_categorical_accuracy: 0.5853
Epoch 6/15
7200/7200 [==============================] - 2362s 328ms/step - loss: 1.0651 - categorical_accuracy: 0.5866 - val_loss: 1.11

In [8]:
model.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 1400)        0           ['input_1[0][0]']                
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 1400, 64)     3200000     ['text_vectorization[0][0]']     
                                                                                                  
 bidirectional (Bidirectional)  (None, 1400, 128)    49920       ['embedding[0][0]']          